In [15]:
import pandas as pd
import numpy
import json
import statistics as s
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import re
from collections import OrderedDict


In [16]:
header_df = pd.read_json('~/WWData/money_diaries/data/raw_basic_info.json')
diary_df = pd.read_json('~/WWData/money_diaries/data/raw_diaries.json')

In [17]:
header_df = header_df.T

In [18]:
cols = [x for x in header_df.columns if len(x) <= 100 ]
#for col in cols:
    #print(col)


In [19]:
df = header_df.loc[:, header_df.isnull().mean() < .8]

In [20]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(cols)

true_k = 10
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [ ]:
for i in range(true_k):
    print(f'Cluster {i}'),
    for ind in order_centroids[i, :20]:
        print(f' {terms[ind]}')

In [22]:
categories = OrderedDict()

categories['income'] = ['paycheck','salary','income']
categories['investments'] = ['401 k','401','401k','roth','ira','savings','investments','retirement']

categories['insurance'] = ['insurance']
categories['health'] = ['hsa','fsa','dental','therapy','medication','disability','vision','medical','health']
categories['home'] =  ['mortgage','renter s insurance','rent','renters','property tax','hoa','housing','home']
categories['utilities'] =  ['internet','wifi','electric','electricity','water','cable','utility','utilities']
categories['car'] = ['auto','fuel','car']
categories['entertainment'] = ['netflix','hulu','hbo','amazon prime','apple','disney','spotify']
categories['debt']  =  ['credit card','student loan','loans','debt']
categories['pet'] = ['cat','dog','pet']



In [23]:
training_data = {}
for col in cols :
    col = re.sub('\W+',' ', col ).lower()
    training_data[col] = None
    for k,v in categories.items():
        for tag in v :
            if re.search(rf"\w?\b{tag}\b\w?",col): 
                training_data[col] = k         
        
    if training_data[col] is None :
        training_data[col] = 'other'


In [24]:
unlabeled_data = { k: v for k,v in training_data.items() if v =='other' }
labeled_data = { k: v for k,v in training_data.items() if v !='other' }

In [ ]:
labeled_data